In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
df1=pd.read_csv("binance-futures_quotes_2023-04-28_BTCBUSD.csv.gz")
df1=df1.sort_values(by='timestamp').drop(["local_timestamp", "exchange", "symbol"], axis=1)
df1

,timestamp,ask_amount,ask_price,bid_price,bid_amount
0,1682640000253000,9.465,29474.5,29474.4,6.426
1,1682640005263000,9.021,29474.5,29474.4,6.426
2,1682640005357000,9.025,29474.5,29474.4,5.994
3,1682640005565000,8.692,29474.5,29474.4,5.994
4,1682640005608000,8.691,29474.5,29474.4,4.994
...,...,...,...,...,...
821732,1682726399205000,2.987,29307.4,29307.3,9.475
821733,1682726399247000,2.835,29307.4,29307.3,9.475
821734,1682726399280000,2.835,29307.4,29307.3,9.478
821735,1682726399300000,2.817,29307.4,29307.3,9.478


In [3]:
df2=pd.read_csv("binance-futures_trades_2023-04-28_BTCBUSD.csv.gz")
df2.sort_values(by='timestamp')

,exchange,symbol,timestamp,local_timestamp,id,side,price,amount
0,binance-futures,BTCBUSD,1682640000016000,1682640000032531,438637782,buy,29474.5,0.132
1,binance-futures,BTCBUSD,1682640000016000,1682640000032539,438637783,buy,29474.5,0.138
2,binance-futures,BTCBUSD,1682640000016000,1682640000032539,438637784,buy,29474.5,0.900
3,binance-futures,BTCBUSD,1682640000016000,1682640000032556,438637785,buy,29474.5,0.138
4,binance-futures,BTCBUSD,1682640000016000,1682640000032557,438637786,buy,29474.5,0.039
...,...,...,...,...,...,...,...,...
497373,binance-futures,BTCBUSD,1682726398979000,1682726398985805,439135158,buy,29307.4,0.001
497375,binance-futures,BTCBUSD,1682726398979000,1682726398985811,439135160,buy,29307.4,0.561
497371,binance-futures,BTCBUSD,1682726398979000,1682726398985685,439135156,buy,29307.4,0.288
497372,binance-futures,BTCBUSD,1682726398979000,1682726398985790,439135157,buy,29307.4,0.001


In [4]:
df2[['timestamp', 'amount']].groupby('timestamp').sum()

,amount
timestamp,
1682640000016000,1.696
1682640000174000,0.002
1682640000190000,0.001
1682640005270000,0.001
1682640005273000,1.000
...,...
1682726394611000,0.079
1682726398957000,3.301
1682726398958000,1.865


In [5]:
df3 = (df2[['timestamp', 'amount']].groupby('timestamp').sum()).join(df2[['timestamp', 'price']].groupby('timestamp').mean()).reset_index()
df3['amount'] = df3['amount'].cumsum()
df3

,timestamp,amount,price
0,1682640000016000,1.696,29474.5
1,1682640000174000,1.698,29474.5
2,1682640000190000,1.699,29474.4
3,1682640005270000,1.700,29474.5
4,1682640005273000,2.700,29474.4
...,...,...,...
152173,1682726394611000,45850.762,29307.3
152174,1682726398957000,45854.063,29307.4
152175,1682726398958000,45855.928,29307.4
152176,1682726398962000,45856.424,29307.4


In [6]:
from datetime import datetime
ts = df1['timestamp'][0]/10**6

# if you encounter a "year is out of range" error the timestamp
# may be in milliseconds, try `ts /= 1000` in that case
print(datetime.utcfromtimestamp(ts).strftime('%Y-%m-%d %H:%M:%S.%f'))

2023-04-28 00:00:00.253000


In [7]:
pd.to_datetime(df1['timestamp']/1e3, unit='ms')

0        2023-04-28 00:00:00.252999936
1        2023-04-28 00:00:05.263000064
2        2023-04-28 00:00:05.356999936
3        2023-04-28 00:00:05.564999936
4        2023-04-28 00:00:05.608000000
                      ...             
821732   2023-04-28 23:59:59.204999936
821733   2023-04-28 23:59:59.247000064
821734   2023-04-28 23:59:59.280000000
821735   2023-04-28 23:59:59.300000000
821736   2023-04-28 23:59:59.656000000
Name: timestamp, Length: 821737, dtype: datetime64[ns]

In [8]:
(df1['timestamp']/1e3).astype('datetime64[ms]')

0        2023-04-28 00:00:00.253
1        2023-04-28 00:00:05.263
2        2023-04-28 00:00:05.357
3        2023-04-28 00:00:05.565
4        2023-04-28 00:00:05.608
                   ...          
821732   2023-04-28 23:59:59.205
821733   2023-04-28 23:59:59.247
821734   2023-04-28 23:59:59.280
821735   2023-04-28 23:59:59.300
821736   2023-04-28 23:59:59.656
Name: timestamp, Length: 821737, dtype: datetime64[ns]

In [9]:
df1.index=pd.to_datetime((df1['timestamp']/1e3).astype('datetime64[ms]').rename('Time'), unit='ms')
df1

,timestamp,ask_amount,ask_price,bid_price,bid_amount
Time,,,,,
2023-04-28 00:00:00.253,1682640000253000,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:05.263,1682640005263000,9.021,29474.5,29474.4,6.426
2023-04-28 00:00:05.357,1682640005357000,9.025,29474.5,29474.4,5.994
2023-04-28 00:00:05.565,1682640005565000,8.692,29474.5,29474.4,5.994
2023-04-28 00:00:05.608,1682640005608000,8.691,29474.5,29474.4,4.994
...,...,...,...,...,...
2023-04-28 23:59:59.205,1682726399205000,2.987,29307.4,29307.3,9.475
2023-04-28 23:59:59.247,1682726399247000,2.835,29307.4,29307.3,9.475
2023-04-28 23:59:59.280,1682726399280000,2.835,29307.4,29307.3,9.478


In [10]:
df3.index=pd.to_datetime((df3['timestamp']/1e3).astype('datetime64[ms]').rename('Time'), unit='ms')
df3

,timestamp,amount,price
Time,,,
2023-04-28 00:00:00.016,1682640000016000,1.696,29474.5
2023-04-28 00:00:00.174,1682640000174000,1.698,29474.5
2023-04-28 00:00:00.190,1682640000190000,1.699,29474.4
2023-04-28 00:00:05.270,1682640005270000,1.700,29474.5
2023-04-28 00:00:05.273,1682640005273000,2.700,29474.4
...,...,...,...
2023-04-28 23:59:54.611,1682726394611000,45850.762,29307.3
2023-04-28 23:59:58.957,1682726398957000,45854.063,29307.4
2023-04-28 23:59:58.958,1682726398958000,45855.928,29307.4


In [11]:
df1 = df1.resample("100ms").ffill().dropna()
df1

,timestamp,ask_amount,ask_price,bid_price,bid_amount
Time,,,,,
2023-04-28 00:00:00.300,1.682640e+15,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:00.400,1.682640e+15,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:00.500,1.682640e+15,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:00.600,1.682640e+15,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:00.700,1.682640e+15,9.465,29474.5,29474.4,6.426
...,...,...,...,...,...
2023-04-28 23:59:59.200,1.682726e+15,2.987,29307.4,29307.3,9.611
2023-04-28 23:59:59.300,1.682726e+15,2.817,29307.4,29307.3,9.478
2023-04-28 23:59:59.400,1.682726e+15,2.817,29307.4,29307.3,9.478


In [12]:
df3 = df3.resample("100ms").ffill().dropna()
df3

,timestamp,amount,price
Time,,,
2023-04-28 00:00:00.100,1.682640e+15,1.696,29474.5
2023-04-28 00:00:00.200,1.682640e+15,1.699,29474.4
2023-04-28 00:00:00.300,1.682640e+15,1.699,29474.4
2023-04-28 00:00:00.400,1.682640e+15,1.699,29474.4
2023-04-28 00:00:00.500,1.682640e+15,1.699,29474.4
...,...,...,...
2023-04-28 23:59:58.500,1.682726e+15,45850.762,29307.3
2023-04-28 23:59:58.600,1.682726e+15,45850.762,29307.3
2023-04-28 23:59:58.700,1.682726e+15,45850.762,29307.3


In [13]:
df1.drop("timestamp", axis=1, inplace=True)
df3.drop("timestamp", axis=1, inplace=True)
df1

,ask_amount,ask_price,bid_price,bid_amount
Time,,,,
2023-04-28 00:00:00.300,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:00.400,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:00.500,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:00.600,9.465,29474.5,29474.4,6.426
2023-04-28 00:00:00.700,9.465,29474.5,29474.4,6.426
...,...,...,...,...
2023-04-28 23:59:59.200,2.987,29307.4,29307.3,9.611
2023-04-28 23:59:59.300,2.817,29307.4,29307.3,9.478
2023-04-28 23:59:59.400,2.817,29307.4,29307.3,9.478


In [14]:
df = df1.join(df3).dropna().rename(columns={
    'ask_price': 'BestAsk',
    'bid_price': 'BestBid',
    'ask_amount': 'AskVol',
    'bid_amount': 'BidVol',
    'amount':'Volume',
    'price':'Price'
})
df

,AskVol,BestAsk,BestBid,BidVol,Volume,Price
Time,,,,,,
2023-04-28 00:00:00.300,9.465,29474.5,29474.4,6.426,1.699,29474.4
2023-04-28 00:00:00.400,9.465,29474.5,29474.4,6.426,1.699,29474.4
2023-04-28 00:00:00.500,9.465,29474.5,29474.4,6.426,1.699,29474.4
2023-04-28 00:00:00.600,9.465,29474.5,29474.4,6.426,1.699,29474.4
2023-04-28 00:00:00.700,9.465,29474.5,29474.4,6.426,1.699,29474.4
...,...,...,...,...,...,...
2023-04-28 23:59:58.500,7.633,29307.4,29307.3,6.528,45850.762,29307.3
2023-04-28 23:59:58.600,7.633,29307.4,29307.3,6.558,45850.762,29307.3
2023-04-28 23:59:58.700,7.633,29307.4,29307.3,6.528,45850.762,29307.3


In [15]:
df[df["BestAsk"]-df["BestBid"]<0]

,AskVol,BestAsk,BestBid,BidVol,Volume,Price
Time,,,,,,


In [16]:
df['Turnover']=df['Price']*df['Volume']
df

,AskVol,BestAsk,BestBid,BidVol,Volume,Price,Turnover
Time,,,,,,,
2023-04-28 00:00:00.300,9.465,29474.5,29474.4,6.426,1.699,29474.4,5.007701e+04
2023-04-28 00:00:00.400,9.465,29474.5,29474.4,6.426,1.699,29474.4,5.007701e+04
2023-04-28 00:00:00.500,9.465,29474.5,29474.4,6.426,1.699,29474.4,5.007701e+04
2023-04-28 00:00:00.600,9.465,29474.5,29474.4,6.426,1.699,29474.4,5.007701e+04
2023-04-28 00:00:00.700,9.465,29474.5,29474.4,6.426,1.699,29474.4,5.007701e+04
...,...,...,...,...,...,...,...
2023-04-28 23:59:58.500,7.633,29307.4,29307.3,6.528,45850.762,29307.3,1.343762e+09
2023-04-28 23:59:58.600,7.633,29307.4,29307.3,6.558,45850.762,29307.3,1.343762e+09
2023-04-28 23:59:58.700,7.633,29307.4,29307.3,6.528,45850.762,29307.3,1.343762e+09


In [17]:
 # Specify naming convention
convention = {
              "BestBid":"BidDiff", 
              "BidVol":"BVolDiff",
              "BestAsk": "AskDiff",
              "AskVol": "AVolDiff",
              "Turnover": "TurnDiff",
              "Volume": "VolDiff"
                }
l, d = 3, 7

In [18]:
# Calculating first diferrences of various columns and dropping null rows
ldata = df[["BestBid", "BidVol", "BestAsk", "AskVol", "Turnover", "Volume"]].diff().rename(columns=convention)
ldata[["BidVol", "AskVol", "Price"]] = df[["BidVol", "AskVol", "Price"]]
ldata["MidPrice"] = (df["BestAsk"] + df["BestBid"])/2
    
# Dealing with a inverted market by straightening it up and weighting it, while weighting no spread as one tenth the tick size
ldata["Spread"] = df["BestAsk"] - df["BestBid"]
    
# Calculating Average of MidPrice for (t,t-1) to be used while calculating MPB
ldata["AvgMP"] = (ldata["MidPrice"] + ldata["MidPrice"].shift(1))/2
    
# Drop first row
ldata.drop(ldata.index[0], inplace=True)
ldata

,BidDiff,BVolDiff,AskDiff,AVolDiff,TurnDiff,VolDiff,BidVol,AskVol,Price,MidPrice,Spread,AvgMP
Time,,,,,,,,,,,,
2023-04-28 00:00:00.400,0.0,0.00,0.0,0.0,0.0,0.0,6.426,9.465,29474.4,29474.45,0.1,29474.45
2023-04-28 00:00:00.500,0.0,0.00,0.0,0.0,0.0,0.0,6.426,9.465,29474.4,29474.45,0.1,29474.45
2023-04-28 00:00:00.600,0.0,0.00,0.0,0.0,0.0,0.0,6.426,9.465,29474.4,29474.45,0.1,29474.45
2023-04-28 00:00:00.700,0.0,0.00,0.0,0.0,0.0,0.0,6.426,9.465,29474.4,29474.45,0.1,29474.45
2023-04-28 00:00:00.800,0.0,0.00,0.0,0.0,0.0,0.0,6.426,9.465,29474.4,29474.45,0.1,29474.45
...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-28 23:59:58.500,0.0,0.00,0.0,0.0,0.0,0.0,6.528,7.633,29307.3,29307.35,0.1,29307.35
2023-04-28 23:59:58.600,0.0,0.03,0.0,0.0,0.0,0.0,6.558,7.633,29307.3,29307.35,0.1,29307.35
2023-04-28 23:59:58.700,0.0,-0.03,0.0,0.0,0.0,0.0,6.528,7.633,29307.3,29307.35,0.1,29307.35


In [19]:
# Calculating Mid Price Change for given delays
ldata["MPC"] = ldata["MidPrice"].shift(-1).rolling(d).mean().shift(1-d) - ldata["MidPrice"]
    
# Calculating Mid Price Basis
ldata["MPB"] = np.where(ldata["VolDiff"] != 0, ((ldata['TurnDiff']/ldata['VolDiff'])), np.nan)
ldata.iloc[0, ldata.columns.get_loc("MPB")] = ldata.iloc[0, ldata.columns.get_loc("MidPrice")]
ldata["MPB"] = ldata["MPB"].fillna(method='ffill')
ldata["MPB"] = ldata["MPB"] - ldata["AvgMP"]
    
# Calculating OIR
ldata["OIR_(t)"] = (ldata["BidVol"] - ldata["AskVol"])/(ldata["BidVol"] + ldata["AskVol"])
    
# Calculating VOI
dBid = pd.Series(np.where(ldata["BidDiff"] < 0, 0, 
                              np.where(ldata["BidDiff"] == 0, ldata["BVolDiff"], ldata["BidVol"])), index=ldata.index)
dAsk = pd.Series(np.where(ldata["AskDiff"] < 0, ldata["AskVol"], 
                              np.where(ldata["AskDiff"] == 0, ldata["AVolDiff"], 0)), index=ldata.index)
ldata["VOI_(t)"] = dBid - dAsk
    
    # Calculating OIR and VOI for each lag by shifting data
for i in range(1, l+1):
    ldata[f"OIR_(t-{i})"] = ldata["OIR_(t)"].shift(i)
    ldata[f"VOI_(t-{i})"] = ldata["VOI_(t)"].shift(i)
        
    # Dropping irrelevant columns
ldata = ldata.drop(columns=ldata.columns[:8])
ldata = ldata.drop(columns=["AvgMP"])
ldata.dropna()

,Price,MidPrice,Spread,MPC,MPB,OIR_(t),VOI_(t),OIR_(t-1),VOI_(t-1),OIR_(t-2),VOI_(t-2),OIR_(t-3),VOI_(t-3)
Time,,,,,,,,,,,,,
2023-04-28 00:00:00.700,29474.4,29474.45,0.1,0.000000e+00,0.000000,-0.191240,0.000,-0.191240,0.0,-0.191240,0.0,-0.191240,0.0
2023-04-28 00:00:00.800,29474.4,29474.45,0.1,3.637979e-12,0.000000,-0.191240,0.000,-0.191240,0.0,-0.191240,0.0,-0.191240,0.0
2023-04-28 00:00:00.900,29474.4,29474.45,0.1,3.637979e-12,0.000000,-0.191240,0.000,-0.191240,0.0,-0.191240,0.0,-0.191240,0.0
2023-04-28 00:00:01.000,29474.4,29474.45,0.1,0.000000e+00,0.000000,-0.191240,0.000,-0.191240,0.0,-0.191240,0.0,-0.191240,0.0
2023-04-28 00:00:01.100,29474.4,29474.45,0.1,3.637979e-12,0.000000,-0.191240,0.000,-0.191240,0.0,-0.191240,0.0,-0.191240,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2023-04-28 23:59:57.800,29307.3,29307.35,0.1,0.000000e+00,-58038.889244,-0.077347,0.000,-0.077347,0.0,-0.077347,0.0,-0.077347,0.0
2023-04-28 23:59:57.900,29307.3,29307.35,0.1,0.000000e+00,-58038.889244,-0.077347,0.000,-0.077347,0.0,-0.077347,0.0,-0.077347,0.0
2023-04-28 23:59:58.000,29307.3,29307.35,0.1,0.000000e+00,-58038.889244,-0.077347,0.000,-0.077347,0.0,-0.077347,0.0,-0.077347,0.0


In [20]:
max(ldata["OIR_(t-3)"].dropna())

0.9999408633944413